In [69]:
#데이터 불러오기 
import pandas as pd
boodongsan_data = pd.read_csv('../final_data/boodongsan_data_utf.csv', encoding='utf-8')
boodongsan_data.head()

#데이터 전처리 

#소재지 없는거 제거 
dataframe = pd.DataFrame()

#중복값 제거 
boodongsan_data = boodongsan_data.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
print(len(boodongsan_data))
#소재지 없는곳 제거
boodongsan_data = boodongsan_data[~boodongsan_data['소재지'].isnull()]
print(len(boodongsan_data))


887
684


In [70]:
#보증금, 월세, 관리비 합치기 
import re
def deposit(x):
    if '월세' in x:
        wol = x.split('/')[0]
        wol = re.sub(r"[^0-9]", "", wol)
        return int(wol)
    elif '전세' in x:
        jun = x.split('(')[0]
        uk = 0
        if '억' in jun:
            uk = jun.split('억')[0]
            jun = jun.split('억')[1]
            uk = re.sub(r"[^0-9]", "", uk)
            uk = int(uk)
        jun = re.sub(r"[^0-9]", "", jun)
        if not jun:
            jun = 0
        return uk*10000 + int(jun)

def monthly(x):
    if '월세' in x:
        if '(' in x:
            x = x.split('(')[0]

        x = x.split('/')[1][:4]
        x = re.sub(r"[^0-9]", "", x)
    else:
        x = 0
    
    return int(x)
def cost(x):
    x = re.sub(r"[^0-9]", "", x)

    #관리비 이상치 처리 1이하인값  처리 
    if int(x) < 1 :
        x = x[-2:] 
    return int(x)

#월세 전세 여부
def charter_montly(x):
    if '월세' in x:
        k = '월세'
    elif '전세' in x:
        k = '전세'
    return k

boodongsan_data['보증금'] = boodongsan_data['가격'].apply(lambda x : deposit(x))
boodongsan_data['월전세'] = boodongsan_data['가격'].apply(lambda x : charter_montly(x))
#보증금 1년 이자 5%
boodongsan_data['보증금_5'] = boodongsan_data['보증금'] * (0.05/12)
boodongsan_data['월세'] = boodongsan_data['가격'].apply(lambda x : monthly(x))
boodongsan_data['월관리비'] = boodongsan_data['월관리비'].apply(lambda x : cost(x))
#총 월비용 
boodongsan_data['월비용'] = boodongsan_data['보증금_5'] + boodongsan_data['월세'] + boodongsan_data['월관리비']

In [71]:
#전용면적 구하기  상가건물은 계약/면적으로 되어있다 
boodongsan_data['전용면적']  = boodongsan_data['계약/전용면적'].fillna('') + boodongsan_data['공급/전용면적'].fillna('')
def area(x):
    x = x.split('㎡')[1].lstrip('/')
    return float(x)
#전용면적 추출
boodongsan_data['전용면적']  = boodongsan_data['전용면적'] .apply(area)

In [72]:
# 좌표 만드는 코드 // 주소만드는 코드  동까지만 표시
def dong_data(x):
    x = ' '.join(x.split()[0:3])
    return x

boodongsan_data['시구동'] =  boodongsan_data['소재지'].apply(dong_data)

In [75]:
import plotly.express as px
# x 전용 면적, y 월비용, color 위치, symbol 월세, 전세, size 보증금
fig = px.scatter(boodongsan_data, x = '월비용',
                 y = '전용면적',
                 color = '시구동', 
                 color_discrete_sequence=px.colors.qualitative.Light24,
                 size = '보증금',
                 hover_name='매물번호',
                 hover_data=['소재지'],
                 symbol= '월전세')

# fig.update_traces(hovertemplate = '%{시구동}')
fig.show()

In [76]:
#저장
import plotly.io as po
po.write_html(fig, file='test.html', auto_open=True)

In [157]:
# 공인 중개사 찾아보기
# 전용면적 20 이상 월 60이하 
broker = boodongsan_data[(boodongsan_data['전용면적'] > 20) & (boodongsan_data['월비용'] < 60)]
broker_count = pd.value_counts(broker['중개사'], sort=True)
broker_count_data = pd.DataFrame()
broker_count_data['raw'] = broker_count.index
broker_count_data['count'] = broker_count.values
broker_count_data

,raw,count
0,거송부동산공인중개사사무소길찾기대표서태석등록번호11230-2019-00153소재지서울...,8
1,한빛부동산공인중개사사무소길찾기대표이경생등록번호11260-2021-00110소재지서울...,8
2,늘봄공인중개사사무소길찾기대표한은혜등록번호1162020190229소재지서울특별시 관악...,7
3,상봉역7번출구부동산 공인중개사무소톡톡문의길찾기대표이경희등록번호11260-2022-0...,4
4,타임공인중개사사무소길찾기대표박정수등록번호11260-2017-00081소재지서울특별시...,3
...,...,...
60,성공공인중개사사무소길찾기대표김수환등록번호11260-2018-00081소재지서울특별시...,1
61,대박공인중개사사무소길찾기대표최준원등록번호11260-2021-00077\t소재지서울특...,1
62,우리공인중개사사무소길찾기대표박은숙등록번호11260-2021-00120소재지서울특별시...,1
63,괜찮은공인중개사사무소톡톡문의길찾기대표원소희등록번호11260-2020-00097소재지...,1


In [158]:
# 전처리 
# 사무소, 위치
def joongaeso(x):
    samuso = ''
    if '길찾기' in x:
        samuso = x.split('길찾기')[0]
    elif '사무소' in x:
        samuso = x.split('사무소')[0]
    sojaeje = x.split('소재지')[1].split('전화')[0]

    return [samuso,sojaeje]

broker_count_data['사무소'],broker_count_data['위치']  = zip(*broker_count_data['raw'].apply(lambda x : joongaeso(x)))

In [163]:
import plotly.express as px
#가지고 있는거 5개 이상만 추출하여 막대 그래프를 만들어준다 
fig = px.bar(broker_count_data[broker_count_data['count'] > 3], x = '사무소', y = 'count', color='위치')
fig.show()

In [164]:
import plotly.express as px
# x 전용 면적, y 월비용, color 위치, symbol 월세, 전세, size 보증금
fig = px.scatter(boodongsan_data[boodongsan_data['중개사'].str.contains('늘봄|한빛|거송|7번출구')], x = '월비용',
                 y = '전용면적',
                 color = '시구동', 
                 color_discrete_sequence=px.colors.qualitative.Light24,
                 size = '보증금',
                 hover_name='매물번호',
                 hover_data=['소재지'],
                 symbol= '월전세')

# fig.update_traces(hovertemplate = '%{시구동}')
fig.show()

In [ ]:
#데이터 시각화 완성 링크로 바로 쉽게 가는 방법 개선하면 될듯 -> 링크 + 좌표값 + 매물번호